In [1]:
import os
dataset_path = "/kaggle/input/shrimp-disease-data"

# List files to confirm it's loaded
print(os.listdir(dataset_path))


['Healthy_train', 'white_spots_train', 'Black_gills_train', 'Black_spots_train']


In [2]:
!pip install ultralytics
!pip install streamlit


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 922.1/922.1 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 98.1 MB/s eta 0:00:00


In [3]:
!pip install ultralytics


In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# Enable mixed precision (optional, speeds up training)
tf.keras.mixed_precision.set_global_policy("mixed_float16")

# Set Dataset Path (Kaggle)
dataset_path ="/kaggle/input/datashrimp"

# Image Data Generator (Data Augmentation)
datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    validation_split=0.2  # 80-20 train-test split
)

# Load Data (Kaggle format)
img_size = (224, 224)
batch_size = 32

train_data = datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="training"
)

val_data = datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation"
)

# Load Pretrained ResNet50 (Feature Extraction)
base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze all layers initially

# Add Custom Classification Layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation="relu")(x)
x = Dense(256, activation="relu")(x)
output_layer = Dense(4, activation="softmax")(x)  # 4 disease classes

# Define Model
model = Model(inputs=base_model.input, outputs=output_layer)

# Compile Model (Initial Training)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001, clipnorm=1.0)  # Gradient Clipping
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

from sklearn.utils.class_weight import compute_class_weight
import numpy as np

class_weights = compute_class_weight(class_weight='balanced',
                                     classes=np.unique(train_data.classes),
                                     y=train_data.classes)
class_weights = dict(enumerate(class_weights))

# Callbacks
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5, verbose=1)
early_stop = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

# Train Model (Initial Feature Extraction Phase)
epochs_initial = 50
model.fit(train_data, validation_data=val_data, epochs=epochs_initial, class_weight=class_weights, callbacks=[reduce_lr, early_stop])

# Fine-Tuning Phase: Unfreeze Last Few Layers
for layer in base_model.layers[-20:]:  # Unfreeze last 20 layers
    layer.trainable = True

# Recompile with a Lower Learning Rate for Fine-Tuning
fine_tune_lr = 1e-5
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=fine_tune_lr, clipnorm=1.0),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

# Train Model (Fine-Tuning Phase)
epochs_fine_tune = 50
model.fit(train_data, validation_data=val_data, epochs=epochs_fine_tune, class_weight=class_weights, callbacks=[reduce_lr, early_stop])

# Save Final Model
model.save("/kaggle/working/shrimp_disease_resnet_finetuned.h5")
print("Fine-tuning completed & model saved!")


Found 4821 images belonging to 4 classes.
Found 1204 images belonging to 4 classes.
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


151/151 ━━━━━━━━━━━━━━━━━━━━ 74s 366ms/step - accuracy: 0.2955 - loss: 1.3867 - val_accuracy: 0.2317 - val_loss: 1.4121 - learning_rate: 1.0000e-04
Epoch 2/50
151/151 ━━━━━━━━━━━━━━━━━━━━ 21s 135ms/step - accuracy: 0.3439 - loss: 1.3323 - val_accuracy: 0.3048 - val_loss: 1.3996 - learning_rate: 1.0000e-04
Epoch 3/50
151/151 ━━━━━━━━━━━━━━━━━━━━ 22s 137ms/step - accuracy: 0.3745 - loss: 1.3050 - val_accuracy: 0.2691 - val_loss: 1.4495 - learning_rate: 1.0000e-04
Epoch 4/50
151/151 ━━━━━━━━━━━━━━━━━━━━ 21s 133ms/step - accuracy: 0.3864 - loss: 1.2997 - val_accuracy: 0.3115 - val_loss: 1.3907 - learning_rate: 1.0000e-04
Epoch 5/50
151/151 ━━━━━━━━━━━━━━━━━━━━ 22s 141ms/step - accuracy: 0.4040 - loss: 1.2622 - val_accuracy: 0.2824 - val_loss: 1.3779 - learning_rate: 1.0000e-04
Epoch 6/50
151/151 ━━━━━━━━━━━━━━━━━━━━ 22s 139ms/step - accuracy: 0.4332 - loss: 1.2371 - val_accuracy: 0.3032 - val_loss: 1.3860 - learning_rate: 1.0000e-04
Epoch 7/50
151/151 ━━━━━━━━━━━━━━━━━━━━ 21s 132ms/step - 